In [39]:
from libsbml import * 
reader = SBMLReader()
document = reader.readSBMLFromFile('created_PA14_model_polisher_output.xml')
model = document.getModel()

In [40]:
print('Species:', model.getNumSpecies())
print('Reactions: ', model.getNumReactions())
print('Compartments: ', model.getNumCompartments())

Species: 1295
Reactions:  1518
Compartments:  4


In [41]:
reaction_list = model.getListOfReactions()

In [42]:
for react in reaction_list:
    react.setId(react.getMetaId())
    #print(react.getId())

In [43]:
metabolite_list = model.getListOfSpecies()

In [44]:
for metabolite in metabolite_list:
    metabolite.setId(metabolite.getMetaId())

### Set objective function

In [45]:
for i in range(len(reaction_list)):
    react = reaction_list[i]
    if 'PA14' in react.getId():
        print(i, react.getId(), react.getMetaId())
print(reaction_list[1485].getId())

1485 R_PA14_Biomass R_PA14_Biomass
R_PA14_Biomass


In [46]:
mplugin = model.getPlugin('fbc')
objective = mplugin.getActiveObjective()
fluxObjective = objective.getFluxObjective(0)
##print(fluxObjective)
print(fluxObjective.getReaction())
print(fluxObjective.getCoefficient())
fluxObjective.setReaction('R_PA14_Biomass')
print(fluxObjective.getReaction())

R_PA14_BIOMASS
1.0
R_PA14_Biomass


### Solving the 4 compartments

In [47]:
changed = 0
for metabolite in metabolite_list:
    if metabolite.getCompartment() =='cytosol':
        metabolite.setCompartment('c')
        changed += 1
    elif metabolite.getCompartment() == 'extracellular':
        metabolite.setCompartment('e')
        changed += 1
#metabolite.setCompartment("cytosol")
print(changed)

868


In [56]:
compartment_list = model.getListOfCompartments()
for comp in compartment_list:
    #print(comp.getId())
    if comp.getId() == 'cytosol':
        comp.removeFromParentAndDelete()
        #print('found')
    elif comp.getId() == 'extracellular':
        comp.removeFromParentAndDelete()

In [52]:
model.getNumCompartments()

2

### Export the SBML file

In [57]:
newdocument = model.getSBMLDocument()
writeSBMLToFile(newdocument,'created_PA14_after_model_polisher.xml') # 1 means success, 0 means failure

1

# Testing the model with COBRA

In [58]:
import cobra

In [59]:
PA14_model =  cobra.io.read_sbml_model('created_PA14_after_model_polisher.xml')
PA14_model.optimize().objective_value

18.95808787453786